# Лаба 5: Определение принадлежности текстов к заданной тематике

In [ ]:
import numpy as np

from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn

In [ ]:
from pymystem3 import Mystem
m = Mystem()

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('[A-Za-zА-Яа-яёЁ]+')

In [ ]:
#attention - generator
def tokenize(text):
    tokens = tokenizer.tokenize(text)
    for i in tokens:
        if len(i)>2:
            yield m.lemmatize(i)[0]

### Данные из ЛК

In [ ]:
#ids of texts you need to classify
ids=[2561, 514, 2570, 3085, 1040, 1046, 24, 3610, 3612, 859, 33, 945, 2087, 1885, 1580, 1074, 3639, 1593, 2485, 573, 63, 3649, 3139, 582, 2049, 1608, 1098, 1099, 2639, 3668, 1626, 607, 2656, 1121, 1635, 3178, 1644, 2158, 2159, 2238, 3699, 3700, 2682, 2173, 2174, 3203, 2692, 2184, 2187, 536, 3213, 3214, 3728, 3729, 658, 2195, 3222, 2199, 2200, 624, 672, 161, 1698, 3748, 711, 3239, 3244, 1710, 176, 3763, 1396, 1717, 695, 698, 1723, 1725, 702, 1215, 1217, 1320, 3267, 196, 710, 3783, 3272, 715, 2255, 724, 1758, 3808, 3300, 3818, 748, 3822, 751, 2288, 3180, 3317, 2806, 3831, 384, 623, 3030, 3842, 3850, 2316, 783, 2324, 1305, 2332, 1311, 1825, 292, 1831, 562, 812, 2863, 3376, 307, 820, 3551, 2361, 833, 2374, 2377, 1355, 3920, 337, 2386, 1367, 1308, 347, 861, 352, 3473, 3940, 870, 2921, 2924, 1524, 13, 2758, 372, 1909, 887, 888, 382, 1174, 897, 3137, 903, 1419, 579, 1006, 1937, 917, 406, 2456, 414, 2463, 3489, 668, 2981, 429, 2760, 1457, 3508, 3509, 1439, 2634, 3001, 3516, 445, 2379, 2496, 1697, 971, 972, 2510, 464, 2003, 1956, 3542, 2521, 2010, 1500, 2526, 479, 3041, 482, 2532, 485, 2023, 3564, 3566, 766, 2548, 3576, 3066, 1020]
texts_index = list(zip(range(200), ids))

In [ ]:
len(ids), len(texts_index)

### Данные тексты

In [ ]:
texts = []
for i in range(1,21):
    #f = codecs.open('/data/lab05data/data/base_'+str(i)+'.txt', 'r', 'utf-8')
    f = open('/data/share/lab05data/base_'+str(i)+'.txt', 'r', encoding='utf-8')
    text = f.readlines()
    texts.append(BeautifulSoup(text[0], "lxml").text.lower())
len(texts)

In [ ]:
for i in ids:
    #f = codecs.open('/data/lab05data/data/test_'+str(i)+'.txt', 'r', 'utf-8')
    f = open('/data/share/lab05data/test_'+str(i)+'.txt', 'r', encoding='utf-8')
    text = f.readlines()
    texts.append(BeautifulSoup(text[0], "lxml").text.lower())
len(texts)

In [ ]:
print("\n".join(texts[:5]))

### Попробуем собственный токенайзер

In [ ]:
from pymystem3 import Mystem
m = Mystem()

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('[A-Za-zА-Яа-яёЁ]+')

In [ ]:
#attention - generator
def tokenize(text):
    tokens = tokenizer.tokenize(text)
    for i in tokens:
        if len(i)>2:
            yield m.lemmatize(i)[0]

In [ ]:
stop_words = ["еще", "него", "сказать", "а", "ж", "нее", "со", "без", "же", "ней", "совсем", "более", "жизнь", 
              "нельзя", "так", "больше", "за", "нет", "такой", "будет", "зачем", "ни", "там", "будто", "здесь", 
              "нибудь", "тебя", "бы", "и", "никогда", "тем", "был", "из", "ним", "теперь", "была", "из", "за",
              "них", "то", "были", "или", "ничего", "тогда", "было", "им", "но", "того", "быть", "иногда", "ну", 
              "тоже", "в", "их", "о", "только", "вам", "к", "об", "том", "вас", "кажется", "один", "тот", "вдруг",
              "как", "он", "три", "ведь", "какая", "она", "тут", "во", "какой", "они", "ты", "вот", "когда", "опять",
              "у", "впрочем", "конечно", "от", "уж", "все", "которого", "перед", "уже", "всегда", "которые", "по",
              "хорошо", "всего", "кто", "под", "хоть", "всех", "куда", "после", "чего", "всю", "ли", "потом", "человек",
              "вы", "лучше", "потому", "чем", "г", "между", "почти", "через", "где", "меня", "при", "что", "говорил",
              "мне", "про", "чтоб", "да", "много", "раз", "чтобы", "даже", "может", "разве", "чуть", "два", "можно",
              "с", "эти", "для", "мой", "сам", "этого", "до", "моя", "свое", "этой", "другой", "мы", "свою", "этом",
              "его", "на", "себе", "этот", "ее", "над", "себя", "эту", "ей", "надо", "сегодня", "я", "ему", "наконец",
              "сейчас", "если", "нас", "сказал", "есть", "не", "сказала"]

### TFIDF для всех тесктов

In [ ]:
tfidf_vec = TfidfVectorizer(norm=None, 
                            smooth_idf=False, 
                            token_pattern='[A-Za-zА-Яа-яёЁ]+', 
                            stop_words=stop_words)

m_tfidf = tfidf_vec.fit_transform(texts)
matrix_tfidf = m_tfidf.toarray()

In [ ]:
matrix_tfidf.shape

In [ ]:
matrix_tfidf

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
%%time
d = pairwise_distances(matrix_tfidf, metric='cosine')

In [ ]:
d.shape

"Так как текстов заданной тематики несколько, результирующую метрику на неизвестный текст можно получить путем сложения косинусной меры этого текста со всеми эталонными текстами. При этом сумма косинусных мер может оказаться больше единицы."

In [ ]:
d20 = 1 - abs(d[:20][:])

In [ ]:
d20.shape

In [ ]:
y=list(d20.sum(axis=0)[20:])

In [ ]:
plt.plot(sorted(y))

In [ ]:
#С токенайзером

In [ ]:
tfidf_vec = TfidfVectorizer(norm=None, 
                            smooth_idf=False, 
                            tokenizer=tokenize, 
                            stop_words=stop_words)
matrix_tfidf = tfidf_vec.fit_transform(texts).toarray()


In [ ]:
%%time
d = pairwise_distances(matrix_tfidf, metric='cosine')

In [ ]:
d20 = 1 - abs(d[:20][:])
y=list(d20.sum(axis=0)[20:])

In [ ]:
plt.plot(sorted(y))

In [ ]:
print(len(y))
print(sum(y))
print(matrix_tfidf.shape)

In [ ]:
average = sum(y)/200
print(average)

In [ ]:
result = {
    'defined': [],
    'other': []
}

for i in y:
    if i > average:
        result['defined'].append(texts_index[y.index(i)][1])
    else:
        result['other'].append(texts_index[y.index(i)][1])
print(result)

"Выходной файл должен быть расположен в корне вашей директории в файле lab05.json. Чекер будет брать файл именно оттуда."

In [ ]:
import json
with open("lab05.json", "w") as outfile:
    json.dump(result, outfile)

Лабораторная будет считаться пройденной, если метрики:

len(TrueDefined.intersection(YourDefined))/len(YourDefined)

len(TrueDefined.intersection(YourDefined))/len(TrueDefined)

будут больше 0.9.

```
time: 2019-20-04 11:56
result is correct: True
len(TrueDefined.intersection(YourDefined))/len(TrueDefined): 1.0
len(TrueDefined.intersection(YourDefined))/len(YourDefined): 1.0
checker has successfully read all data in file: True
file lab05.json exists in your dir: True
file has size less than 20 000 bytes: True
```